In [1]:
from config.settings import SRC_DIR
import os
os.chdir(SRC_DIR)

In [2]:
from config.settings import DATA_DIR
import os

file_dir = DATA_DIR / 'valid_invoices'
files = os.listdir(file_dir)
file_names = [file[:-4] for file in files if file.endswith('.pdf')]
file_paths = [file_dir/file for file in files if file.endswith('.pdf')]
file_names

['invoice', 'ISS', 'PerfectMatch', 'PO139090 GP']

In [3]:
from tqdm.auto import tqdm
from invoice_processing_system.components.data_ingestion import DataIngestion
from invoice_processing_system.components.data_extraction import DataExtraction

pdf_items = []
for pdf, pdf_path in zip(tqdm(file_names),file_paths):

    data_ingester = DataIngestion(pdf_path)
    data_extracter = DataExtraction(data_ingester)
    pages, tables, images = data_extracter.extract_data()
    pdf_items.append(pages)

ModuleNotFoundError: No module named 'invoice_processing_system'

In [ ]:
import os
from invoice_processing_system.components.model_inference import ModelInference

# Define the PDF folder path
pdf_folder = os.path.join(file_dir, 'invoice_output', pdf, 'invoice_14b')

# Ensure the directory exists
os.makedirs(pdf_folder, exist_ok=True)

# Initialize the model
model = ModelInference(model_name="Qwen/Qwen2.5-14B")

INFO:root:Loading model and tokenizer...
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0
INFO:root:Model loaded successfully!


In [8]:
# print(model.get_prompt(all_items[0]))
# Process items
response = model.process_items(pages, output_folder=pdf_folder)

Processing Items:   0%|          | 0/1 [00:00<?, ?item/s]

INFO:root:Saved: C:\Users\ai.users\local_ocr\data\valid_invoices\invoice_output\PO139090 GP\invoice_14b\qc_report_item_0.json
